In [3]:
import psycopg2
from config import config
import pandas as pd

In [4]:
params = config()
connection = psycopg2.connect(**params)

In [110]:
query= """SELECT
    i.item_name,
    COUNT(o.id) AS order_count,
    COUNT(DISTINCT s.id) AS seller_count,
    COUNT(CASE WHEN c.male = 'male' THEN 1 END) AS male_order_count,
    COUNT(CASE WHEN c.male = 'female' THEN 1 END) AS female_order_count,
     (COUNT(CASE WHEN c.male = 'male' THEN 1 END)::float / COUNT(DISTINCT c.id) * 100) AS male_percentage,
    (COUNT(CASE WHEN c.male = 'female' THEN 1 END)::float / COUNT(DISTINCT c.id) * 100) AS female_percentage,
    AVG(g.item_price) AS avg_price,
    MIN(g.item_price) AS min_price,
    MAX(g.item_price) AS max_price
FROM
    "order" o
JOIN
    "item" i ON o.item_id = i.id
JOIN
    "customer" c ON o.customer_id = c.id
JOIN
    "seller" s ON o.seller_id = s.id
JOIN 
    "good" g ON o.good_id = g.id
GROUP BY
    i.item_name
ORDER BY
    order_count DESC
LIMIT 5;
;
"""

In [111]:
df = pd.read_sql(query, con=connection)

C:\Users\Knopka\AppData\Local\Temp\ipykernel_18960\635191612.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


In [112]:
df

,item_name,order_count,seller_count,male_order_count,female_order_count,male_percentage,female_percentage,avg_price,min_price,max_price
0,Camera,100,79,53,47,54.081633,47.959184,5929.100000,1259,9902
1,Cleaning agent,65,58,28,37,45.901639,60.655738,5260.707692,1024,9847
2,Window cleaning solution,65,57,34,31,52.307692,47.692308,5508.615385,1024,9727
3,Headphones,64,51,34,30,54.838710,48.387097,6066.687500,1043,9902
4,Meat,62,51,32,30,54.237288,50.847458,5992.645161,1118,9639


In [131]:
query= """WITH AgeGroups AS (
    SELECT
        CASE
            WHEN age BETWEEN 18 AND 25 THEN '18-25'
            WHEN age BETWEEN 26 AND 35 THEN '26-35'
            WHEN age BETWEEN 36 AND 45 THEN '36-45'
            WHEN age BETWEEN 46 AND 55 THEN '46-55'
            ELSE '56+'
        END AS age_group,
        i.item_name,
        COUNT(o.id) AS purchase_count,
        ROW_NUMBER() OVER (PARTITION BY CASE WHEN age BETWEEN 18 AND 25 THEN '18-25'
                                           WHEN age BETWEEN 26 AND 35 THEN '26-35'
                                           WHEN age BETWEEN 36 AND 45 THEN '36-45'
                                           WHEN age BETWEEN 46 AND 55 THEN '46-55'
                                           ELSE '56+' END
                          ORDER BY COUNT(o.id) DESC) AS item_rank
    FROM
        "customer" c
    JOIN
        "order" o ON c.id = o.customer_id
    JOIN
        "item" i ON o.item_id = i.id
    GROUP BY
        age_group, i.item_name
)

SELECT
    age_group,
    item_name,
    purchase_count
FROM
    AgeGroups
WHERE
    item_rank = 1
ORDER BY
    age_group;


"""

In [132]:
df = pd.read_sql(query, con=connection)

C:\Users\Knopka\AppData\Local\Temp\ipykernel_18960\635191612.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


In [133]:
df

,age_group,item_name,desc
0,18-25,Camera,14
1,26-35,Mosquito aerosol,17
2,36-45,Camera,20
3,46-55,Soap,16
4,56+,Camera,40


In [147]:
query= """WITH PriceCategories AS (
    SELECT
        item_name,
        CASE
            WHEN item_price < 1500 THEN '0-1500'
            WHEN item_price BETWEEN 1500 AND 2999 THEN '1000-2999'
            WHEN item_price BETWEEN 3000 AND 999 THEN '3000-5999'
            WHEN item_price BETWEEN 6000 AND 8999 THEN '6000-8999'
            ELSE '9000+'
        END AS price_category,
        g.item_price,
        COUNT(o.id) AS sales_count
    FROM
        "item" i
    JOIN
        "order" o ON i.id = o.item_id
    JOIN 
        "good" g ON g.id = o.good_id
    GROUP BY
        i.id, item_name, price_category, item_price
)

SELECT
    price_category,
    AVG(item_price) AS average_price,
    PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY item_price) AS median_price,
    SUM(sales_count) AS total_sales
FROM
    PriceCategories
GROUP BY
    price_category
ORDER BY
    price_category;
"""

In [148]:
df = pd.read_sql(query, con=connection)

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\635191612.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


In [149]:
df

,price_category,average_price,median_price,total_sales
0,0-1500,1206.020000,1118.0,289.0
1,1000-2999,2179.775671,2010.0,717.0
2,6000-8999,7446.814318,7471.0,2053.0
3,9000+,6038.940668,5324.0,1941.0


In [150]:
query= """WITH AgeCategories AS (
    SELECT
        c.id AS customer_id,
        CASE
            WHEN age BETWEEN 18 AND 25 THEN '18-25'
            WHEN age BETWEEN 26 AND 35 THEN '26-35'
            WHEN age BETWEEN 36 AND 45 THEN '36-45'
            WHEN age BETWEEN 46 AND 55 THEN '46-55'
            ELSE '56+'
        END AS age_category
    FROM
        "customer" c
)

SELECT
    age_category,
    popular_item_category,
    order_count
FROM (
    SELECT
        ac.age_category,
        i.item_category AS popular_item_category,
        COUNT(o.id) AS order_count,
        ROW_NUMBER() OVER (PARTITION BY ac.age_category ORDER BY COUNT(o.id) DESC) AS rnk
    FROM
        "order" o
    JOIN
        AgeCategories ac ON o.customer_id = ac.customer_id
    JOIN
        "item" i ON o.item_id = i.id
    WHERE
        ac.age_category IS NOT NULL  -- Exclude "56+" category
    GROUP BY
        ac.age_category, i.item_category
) ranked
WHERE
    rnk <= 3 
ORDER BY
    age_category, rnk;




"""

In [151]:
df = pd.read_sql(query, con=connection)

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\635191612.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


In [152]:
df

,age_category,popular_item_category,order_count
0,18-25,Electronics,125
1,18-25,Food and Beverages,123
2,18-25,Clothing,122
3,26-35,Chemistry,174
4,26-35,Electronics,165
5,26-35,Food and Beverages,163
6,36-45,Food and Beverages,165
7,36-45,Electronics,143
8,36-45,Beauty and Health,141
9,46-55,Chemistry,189


In [153]:
query= """WITH OrderTotals AS (
    SELECT
        o.id,
        o.payment_type,
        o.customer_id,
        SUM(g.item_price) AS total_amount
    FROM
        "order" o
    JOIN
        "good" g ON o.item_id = g.item_id
    GROUP BY
        o.id, o.payment_type, o.customer_id
),
CustomerDetails AS (
    SELECT
        c.id AS customer_id,
        c.male,
        c.age
    FROM
        "customer" c
)
SELECT
    ot.payment_type,
    AVG(ot.total_amount) AS avg_order_amount,
    COUNT(ot.id) AS order_count,
    COUNT(DISTINCT ot.customer_id) AS customer_count,
    COUNT(DISTINCT CASE WHEN cd.male = 'male' THEN cd.customer_id END) AS male_count,
    COUNT(DISTINCT CASE WHEN cd.male = 'female' THEN cd.customer_id END) AS female_count,
    (COUNT(DISTINCT CASE WHEN cd.male = 'female' THEN cd.customer_id END) * 100.0) / COUNT(DISTINCT ot.customer_id) AS percentage_female,
    (COUNT(DISTINCT CASE WHEN cd.male = 'male' THEN cd.customer_id END) * 100.0) / COUNT(DISTINCT ot.customer_id) AS percentage_male
FROM
    OrderTotals ot
JOIN
    CustomerDetails cd ON ot.customer_id = cd.customer_id
GROUP BY
    ot.payment_type
ORDER BY
    avg_order_amount DESC;


"""

In [154]:
df = pd.read_sql(query, con=connection)
df

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\487524798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,payment_type,avg_order_amount,order_count,customer_count,male_count,female_count,percentage_female,percentage_male
0,cash payment,13682.659371,1462,781,380,401,51.344430,48.655570
1,cashless payment,13464.354033,1401,760,375,385,50.657895,49.342105
2,in installments,13414.580274,1389,736,352,384,52.173913,47.826087


In [155]:
query= """WITH OrderTotals AS (
    SELECT
        o.id,
        o.customer_id,
        o.seller_id,
        SUM(g.item_price) AS total_amount
    FROM
        "order" o
    JOIN
        "good" g ON o.item_id = g.item_id
    GROUP BY
        o.id, o.customer_id, o.seller_id
),
CustomerDetails AS (
    SELECT
        c.id AS customer_id,
        c.city,
        c.age
    FROM
        "customer" c
)
SELECT
    cd.city,
    COUNT(ot.id) AS order_count,
    cd.age AS avg_customer_age
FROM
    OrderTotals ot
JOIN
    CustomerDetails cd ON ot.customer_id = cd.customer_id
GROUP BY
    cd.city, cd.age
ORDER BY
    order_count DESC
Limit 10;
"""

In [156]:
df = pd.read_sql(query, con=connection)
df

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\487524798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,city,order_count,avg_customer_age
0,Ibarrabury,13,30
1,Samuelfort,12,75
2,Johnmouth,12,30
3,Wolfeburgh,11,21
4,South Lorishire,11,44
5,East Tyler,11,25
6,Port Erin,10,60
7,North Brendaburgh,10,32
8,Robertport,10,59
9,Joelport,10,24


In [157]:
query= """WITH SellerRatings AS (
    SELECT
        s.id AS seller_id,
        s.seller_name,
        AVG(o.seller_raiting) AS avg_seller_rating
    FROM
        "seller" s
    JOIN
        "order" o ON s.id = o.seller_id
    GROUP BY
        s.id, s.seller_name
),
SellerSales AS (
    SELECT
        s.id AS seller_id,
        COUNT(o.id) AS total_sales,
        COUNT(CASE WHEN o.date >= CURRENT_DATE - INTERVAL '7 days' THEN o.id END) AS sales_last_week
    FROM
        "seller" s
    LEFT JOIN
        "order" o ON s.id = o.seller_id
    GROUP BY
        s.id
)
SELECT
    sr.seller_name,
    sr.avg_seller_rating,
    ss.total_sales,
    ss.sales_last_week,
    c.seller_city
FROM
    SellerRatings sr
JOIN
    SellerSales ss ON sr.seller_id = ss.seller_id
JOIN
    "seller" c ON sr.seller_id = c.id
ORDER BY
    avg_seller_rating DESC
LIMIT 10;

"""

In [158]:
df = pd.read_sql(query, con=connection)
df

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\487524798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,seller_name,avg_seller_rating,total_sales,sales_last_week,seller_city
0,"Thompson, Williams and Wilkerson",4.041667,24,4,West Gregory
1,Lyons-Melendez,3.909091,22,6,South Tabithachester
2,Bray LLC,3.666667,27,1,Stokesfurt
3,Dunn Inc,3.642857,28,8,Ericville
4,Pacheco-Adams,3.636364,11,3,West James
5,Green-Lyons,3.560000,25,5,Chenmouth
6,Campbell-Morgan,3.541667,24,7,Anthonybury
7,Beck-Chambers,3.541667,24,5,Gilesburgh
8,"Valentine, Nelson and Smith",3.500000,28,6,Brianbury
9,Patterson LLC,3.458333,24,7,West Williamstad


In [159]:
query= """WITH SellerInfo AS (
    SELECT
        s.id AS seller_id,
        s.seller_name,
        AVG(o.seller_raiting) AS avg_seller_rating,
        COUNT(o.id) AS total_sales,
        COUNT(CASE WHEN o.date >= CURRENT_DATE - INTERVAL '7 days' THEN o.id END) AS sales_last_week,
        c.seller_city,
        MAX(o.payment_type) AS most_popular_payment_method
    FROM
        "seller" s
    LEFT JOIN
        "order" o ON s.id = o.seller_id
    LEFT JOIN
        "seller" c ON s.id = c.id
    GROUP BY
        s.id, s.seller_name, c.seller_city
)
SELECT
    si.seller_name,
    si.avg_seller_rating,
    si.total_sales,
    si.sales_last_week,
    si.most_popular_payment_method,
    si.seller_city
FROM
    SellerInfo si
ORDER BY
    si.avg_seller_rating DESC
LIMIT 10;




"""

In [160]:
df = pd.read_sql(query, con=connection)
df

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\487524798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,seller_name,avg_seller_rating,total_sales,sales_last_week,most_popular_payment_method,seller_city
0,"Thompson, Williams and Wilkerson",4.041667,24,4,in installments,West Gregory
1,Lyons-Melendez,3.909091,22,6,in installments,South Tabithachester
2,Bray LLC,3.666667,27,1,in installments,Stokesfurt
3,Dunn Inc,3.642857,28,8,in installments,Ericville
4,Pacheco-Adams,3.636364,11,3,in installments,West James
5,Green-Lyons,3.560000,25,5,in installments,Chenmouth
6,Campbell-Morgan,3.541667,24,7,in installments,Anthonybury
7,Beck-Chambers,3.541667,24,5,in installments,Gilesburgh
8,"Valentine, Nelson and Smith",3.500000,28,6,in installments,Brianbury
9,Patterson LLC,3.458333,24,7,in installments,West Williamstad


In [161]:
query= """WITH CustomerBonusSum AS (
    SELECT
        c.id AS customer_id,
        c.name AS customer_name,
        c.surname AS customer_surname,
        SUM(o.bonus) AS total_bonus
    FROM
        "customer" c
    LEFT JOIN
        "order" o ON c.id = o.customer_id
    GROUP BY
        c.id, c.name, c.surname
    HAVING
        SUM(o.bonus) IS NOT NULL
)
SELECT
    cbs.customer_name,
    cbs.customer_surname,
    cbs.total_bonus
FROM
    CustomerBonusSum cbs
ORDER BY
    cbs.total_bonus DESC
LIMIT 10;


"""

In [162]:
df = pd.read_sql(query, con=connection)
df

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\487524798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,customer_name,customer_surname,total_bonus
0,Brent,Phillips,443
1,Omar,Davis,384
2,Jeffrey,Cherry,366
3,Cory,Davies,347
4,Erika,Stout,340
5,Ana,Johnson,326
6,Brandon,Andrews,319
7,Deanna,Stephens,313
8,Antonio,Morrison,311
9,Anthony,White,310


In [163]:
query= """WITH CustomerPurchaseDetails AS (
    SELECT
        c.id AS customer_id,
        c.name AS customer_name,
        c.surname AS customer_surname,
        SUM(o.bonus) AS total_bonus,
        SUM(g.item_price) AS total_spent,
        COUNT(DISTINCT o.id) AS total_orders,
        MAX(i.item_category) AS most_purchased_category
    FROM
        "customer" c
    LEFT JOIN
        "order" o ON c.id = o.customer_id
    LEFT JOIN
        "good" g ON o.good_id = g.id
    LEFT JOIN
        "item" i ON o.item_id = i.id
    GROUP BY
        c.id, c.name, c.surname
    HAVING
        SUM(o.bonus) IS NOT NULL
)
SELECT
    cpd.customer_name,
    cpd.customer_surname,
    cpd.total_bonus,
    cpd.total_spent,
    cpd.total_orders,
    cpd.most_purchased_category
FROM
    CustomerPurchaseDetails cpd
ORDER BY
    cpd.total_bonus DESC
LIMIT 10;
"""

In [164]:
df = pd.read_sql(query, con=connection)
df

C:\Users\Knopka\AppData\Local\Temp\ipykernel_2732\487524798.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, con=connection)


,customer_name,customer_surname,total_bonus,total_spent,total_orders,most_purchased_category
0,Brent,Phillips,443,67076,13,Electronics
1,Omar,Davis,384,81142,12,Food and Beverages
2,Jeffrey,Cherry,366,69580,12,Food and Beverages
3,Cory,Davies,347,89558,13,Food and Beverages
4,Erika,Stout,340,72001,10,Electronics
5,Ana,Johnson,326,65066,12,Electronics
6,Brandon,Andrews,319,75535,13,Food and Beverages
7,Deanna,Stephens,313,52465,9,Food and Beverages
8,Antonio,Morrison,311,75119,12,Food and Beverages
9,Anthony,White,310,65781,11,Food and Beverages
